In [2]:
### import shutil
import tqdm
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import numpy as np
import os, sys
import pickle, functools, operator
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import joblib
from tensorflow.keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
import json
import random
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import datetime

from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN , Concatenate
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

import random
random.seed(42)

In [3]:
class config():
    train_path = "./training_data"
    test_path = "./testing_data"
    batch_size = 160
    learning_rate = 0.0007
    epochs = 30
    latent_dim = 512
    num_encoder_tokens = 4096
    num_decoder_tokens = 1500
    time_steps_encoder = 80
    max_probability = -1
    save_model_path = 'model_final' 
    max_length = 30
    search_type = 'greedy'

In [4]:
config = config()

In [5]:
CUDA_VISIBLE_DEVICES="0"

## 02. Train & Test Split Data

In [6]:
with open('./training_data/AllVideoDescriptions.txt') as f:
    lines = f.readlines()

In [7]:
with open('./train_split.txt','r') as f:
    train_l = f.readlines()
train_l = [l.replace('\n','') for l in train_l]

In [8]:
with open('./test_split.txt','r') as f:
    test_l = f.readlines()
test_l = [l.replace('\n','') for l in test_l]

In [9]:
with open('./val_split.txt','r') as f:
    val_l = f.readlines()
val_l = [l.replace('\n','') for l in val_l]

In [10]:
scripts = {}
for l in lines:
    id = l.split(' ')[0]
    script = ' '.join(l.split(' ')[1:])
    if id in scripts:
      scripts[id].append(script)
    else:
      scripts[id] = [script]

In [11]:
training_list = []
validation_list = []
test_list = [] 
vocab_list = []

for y in train_l:
  for caption in scripts[y]:
    caption = "<bos> " + caption + " <eos>"

    training_list.append([caption, y])

for y in val_l:
  for caption in scripts[y]:
    caption = "<bos> " + caption + " <eos>"

    validation_list.append([caption, y])
    
for y in test_l:
  for caption in scripts[y]:
    caption = "<bos> " + caption + " <eos>"

    test_list.append([caption, y])

train_list = training_list + validation_list + test_list

In [12]:
train_list = training_list + validation_list + test_list

In [13]:
print(len(train_list))
random.shuffle(training_list)
random.shuffle(validation_list)
random.shuffle(test_list)

for train in training_list:
    vocab_list.append(train[0])
# Tokenizing the words
tokenizer = Tokenizer(num_words=1500)
tokenizer.fit_on_texts(vocab_list)
print(len(tokenizer.word_index))
x_data = {}
TRAIN_FEATURE_DIR = os.path.join('training_data', 'features_dir')
# Loading all the numpy arrays at once and saving them in a dictionary
for filename in os.listdir(TRAIN_FEATURE_DIR):
    f = np.load(os.path.join(TRAIN_FEATURE_DIR, filename))
    x_data[filename[:-4]] = f
print(len(training_list))
print(len(validation_list))
len(x_data)

80827
9463
48774
4290


1970

## 03. Model Training

In [14]:
# Creating a custom data generator because we cannot load so many files at once
def load_datatest(train_path, epochs=100, x_data=x_data, tokenizer=tokenizer, num_decoder_tokens=1500,training_list=training_list, batch_size=32, maxlen=11):
    encoder_input_data = []
    decoder_input_data = []
    decoder_target_data = []
    videoId = []
    videoSeq = []
    # separating the videoId and the video captions
    for idx, cap in enumerate(training_list):
        caption = cap[0]
        videoId.append(cap[1])
        videoSeq.append(caption)
    # converting the captions to tokens and padding them to equal sizes
    train_sequences = tokenizer.texts_to_sequences(videoSeq)
    train_sequences = np.array(train_sequences)
    train_sequences = pad_sequences(train_sequences, padding='post',truncating='post', maxlen=maxlen)
    max_seq_length = train_sequences.shape[1]
    filesize = len(train_sequences)
    vCount = 0
    n = 0
    for i in range(epochs):
      for idx in  range(0,filesize):
        n += 1
        try:
          encoder_input_data.append(x_data[videoId[idx]])
        except:
          continue
        y = to_categorical(train_sequences[idx], num_decoder_tokens)
        decoder_input_data.append(y[:-1])
        decoder_target_data.append(y[1:])
        if n == batch_size:
          encoder_input = np.array(encoder_input_data)
          decoder_input = np.array(decoder_input_data)
          decoder_target = np.array(decoder_target_data)
          encoder_input_data = []
          decoder_input_data = []
          decoder_target_data = []
          n = 0
          yield ([encoder_input, decoder_input, decoder_target])


In [15]:
# writing the train and validation generator
train = load_datatest(train_path='training_data',batch_size=config.batch_size, training_list=training_list, x_data=x_data, epochs=300)
valid = load_datatest(train_path='training_data',batch_size=config.batch_size, training_list=validation_list, x_data=x_data, epochs=300)
test = load_datatest(train_path='test_data',batch_size=config.batch_size, training_list=test_list, x_data=x_data,epochs=1)

In [16]:
class Encoder(tf.keras.Model):
    def __init__(self, hidden_dim,time_steps_encoder,num_encoder_tokens):
        super(Encoder, self).__init__()
        # Define the RNN layer, LSTM
        self.hidden_dim = hidden_dim
        self.lstm = tf.keras.layers.LSTM( 
            hidden_dim, return_sequences=True, return_state=True, input_shape=(time_steps_encoder,num_encoder_tokens))

    def call(self, input_sequence, states):
        # Call the LSTM unit
        output, state_h, state_c = self.lstm(input_sequence, initial_state=states)

        return output, state_h, state_c

    def init_states(self, batch_size):
        # Return a all 0s initial states
        return (tf.zeros([batch_size, self.hidden_dim]),
                tf.zeros([batch_size, self.hidden_dim]))

In [17]:
class LuongAttention(tf.keras.Model):
    def __init__(self, rnn_size, attention_func):
        super(LuongAttention, self).__init__()
        self.attention_func = attention_func

        if attention_func not in ['dot', 'general', 'concat']:
            raise ValueError(
                'Attention score must be either dot, general or concat.')

        if attention_func == 'general':
            # General score function
            self.wa = tf.keras.layers.Dense(rnn_size)
        elif attention_func == 'concat':
            # Concat score function
            self.wa = tf.keras.layers.Dense(rnn_size, activation='tanh')
            self.va = tf.keras.layers.Dense(1)

    def call(self, decoder_output, encoder_output):
        if self.attention_func == 'dot':
            # Dot score function: decoder_output (dot) encoder_output
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, encoder_output, transpose_b=True) # (batch_size, 1, max_len)
        elif self.attention_func == 'general':
            # General score function: decoder_output (dot) (Wa (dot) encoder_output)
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, self.wa(
                encoder_output), transpose_b=True) #(batch_size, 1, max_len)
        elif self.attention_func == 'concat':
            # Concat score function: va (dot) tanh(Wa (dot) concat(decoder_output + encoder_output))
            # Decoder output must be broadcasted to encoder output's shape first
            decoder_output = tf.tile(
                decoder_output, [1, encoder_output.shape[1], 1]) #shape (batch size, max len,hidden_dim)

            # Concat => Wa => va
            # (batch_size, max_len, 2 * rnn_size) => (batch_size, max_len, rnn_size) => (batch_size, max_len, 1)
            score = self.va(
                self.wa(tf.concat((decoder_output, encoder_output), axis=-1))) # (batch_size, max len, 1)

            # Transpose score vector to have the same shape as other two above
            # (batch_size, max_len, 1) => (batch_size, 1, max_len)
            score = tf.transpose(score, [0, 2, 1]) #(batch_size, 1, max_len)

        # alignment a_t = softmax(score)
        alignment = tf.keras.activations.softmax(score, axis=-1) #(batch_size, 1, max_len)
        
        # context vector c_t is the weighted average sum of encoder output
        context = tf.matmul(alignment, encoder_output) # (batch_size, 1, hidden_dim)

        return context, alignment

In [18]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size,hidden_dim,time_steps_encoder,num_encoder_tokens,attention_func):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.attention = LuongAttention(hidden_dim, attention_func)
        self.lstm = tf.keras.layers.LSTM(
            hidden_dim, return_sequences=True, return_state=True,input_shape=(None,num_encoder_tokens))
        self.wc = tf.keras.layers.Dense(hidden_dim, activation='tanh')
        self.ws = tf.keras.layers.Dense(vocab_size,activation='softmax')
        
    def call(self, input_sequence, state,encoder_output):
        # Call the LSTM unit
        lstm_out, state_h, state_c = self.lstm(input_sequence, state)
        context , alignment = self.attention(lstm_out,encoder_output)
        lstm_out = tf.concat(
        [tf.squeeze(context, 1), tf.squeeze(lstm_out, 1)], 1)
        lstm_out = self.wc(lstm_out)
        logits = self.ws(lstm_out)


        return logits, state_h, state_c, alignment

In [19]:
#Set the length of the input and output vocabulary
time_steps_encoder=80
num_encoder_tokens=4096
latent_dim=512
time_steps_decoder=30
num_decoder_tokens=1500
batch_size=160
HIDDEN_DIM =  latent_dim

#Create the encoder
encoder = Encoder(HIDDEN_DIM,time_steps_encoder,num_encoder_tokens)
# Get the initial states
initial_state = encoder.init_states(1)

# Create the decoder
decoder = Decoder(num_decoder_tokens,HIDDEN_DIM,time_steps_decoder,num_encoder_tokens,'general')


2022-06-09 11:03:28.903513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 11:03:28.904388: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 11:03:28.912480: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 11:03:28.913048: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 11:03:28.913591: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [20]:
def loss_func(targets, logits):
    crossentropy = tf.keras.losses.CategoricalCrossentropy()
    # Mask padding values, they do not have to compute for loss
    mask = tf.math.logical_not(tf.math.equal(tf.argmax(targets,-1), 0))
    mask = tf.cast(mask, dtype=tf.int64)
    # Calculate the loss value
    loss = crossentropy(targets, logits, sample_weight=mask)

    return loss


def accuracy_fn(y_true, y_pred):
    # y_pred shape is batch_size, seq length, vocab size
    # y_true shape is batch_size, seq length
    pred_values = K.cast(K.argmax(y_pred, axis=-1), dtype='int32')
    y_true = K.cast(K.argmax(y_true, axis=-1), dtype='int32')
    correct = K.cast(K.equal(y_true, pred_values), dtype='float32')

    # 0 is padding, don't include those
    mask = K.cast(K.greater(y_true, 0), dtype='float32')
    n_correct = K.sum(mask * correct)
    n_total = K.sum(mask)
    return n_correct / n_total

In [21]:
# Use the @tf.function decorator to take advance of static graph computation
@tf.function
def train_step_att(input_seq, target_seq_in, target_seq_out, en_initial_states, optimizer):
    ''' A training step, train a batch of the data and return the loss value reached
        Input:
        - input_seq: array of integers, shape [batch_size, max_seq_len, embedding dim].
            the input sequence
        - target_seq_out: array of integers, shape [batch_size, max_seq_len, embedding dim].
            the target seq, our target sequence
        - target_seq_in: array of integers, shape [batch_size, max_seq_len, embedding dim].
            the input sequence to the decoder, we use Teacher Forcing
        - en_initial_states: tuple of arrays of shape [batch_size, hidden_dim].
            the initial state of the encoder
        - optimizer: a tf.keras.optimizers.
        Output:
        - loss: loss value
        
    '''
    loss = 0.
    acc = 0.
    logits = None
    
    with tf.GradientTape() as tape:
        en_outputs = encoder(input_seq, en_initial_states)
        en_states = en_outputs[1:]
        de_state_h, de_state_c = en_states

        # We need to create a loop to iterate through the target sequences
        for i in range(target_seq_out.shape[1]):
            # Input to the decoder must have shape of (batch_size, length)
            # so we need to expand one dimension
            decoder_in = tf.expand_dims(target_seq_in[:, i], 1)
            logit, de_state_h, de_state_c, _ = decoder(
                decoder_in, (de_state_h, de_state_c), en_outputs[0])

            # The loss is now accumulated through the whole batch
            loss += loss_func(target_seq_out[:, i], logit)
            # Store the logits to calculate the accuracy
            logit = K.expand_dims(logit, axis=1)
            if logits is None:
                logits = logit
            else:
                logits = K.concatenate((logits,logit), axis=1)
        # Calculate the accuracy for the batch data   
        acc = accuracy_fn(target_seq_out, logits)
    # Update the parameters and the optimizer
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss / target_seq_out.shape[1] ,acc

In [22]:
import time
# Create the main train function
def main_train(encoder, decoder, dataset, n_epochs, batch_size, optimizer, checkpoint, checkpoint_prefix):
    
    losses = []
    accuracies = []
    for e in range(n_epochs):
        # Get the initial time
        start = time.time()
        # Get the initial state for the encoder
        en_initial_states = encoder.init_states(batch_size)
        # For every batch data
        batch = 0
        for (input_seq, target_seq_in, target_seq_out) in dataset:
            # Train and get the loss value 
            batch += 1 
            loss,acc = train_step_att(input_seq, target_seq_in, target_seq_out, en_initial_states, optimizer)
            if batch % 100 == 0:
                # Store the loss and accuracy values
                losses.append(loss)
                accuracies.append(acc)
                print('Epoch {} Batch {} Loss {:.4f} Acc{:.4f}'.format(e + 1, batch, loss.numpy(),acc.numpy()))
                
        # saving (checkpoint) the model every 2 epochs
        if (e + 1) % 2 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
    
        print('Time taken for 1 epoch {:.4f} sec\n'.format(time.time() - start))
        
    return losses

In [23]:
optimizer = tf.keras.optimizers.Adam(clipnorm=5.0,learning_rate=0.001)
# Create a checkpoint object to save the model
checkpoint_dir = './training_ckpt_seq2seq_att_general'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

losses = main_train(encoder, decoder, train, config.epochs, batch_size, optimizer, checkpoint, checkpoint_prefix)


/tmp/ipykernel_2440552/2721968490.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_sequences = np.array(train_sequences)
2022-06-09 11:03:38.126478: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "GeForce RTX 2080 Ti" frequency: 1650 num_cores: 68 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5767168 shared_memory_size_per_multiprocessor: 65536 memory_size: 7769620480 bandwidth: 616000000 } outputs { dtype: 

Epoch 1 Batch 100 Loss 2.5367 Acc0.3948
Epoch 1 Batch 200 Loss 2.1949 Acc0.4247
Epoch 1 Batch 300 Loss 1.9830 Acc0.4655
Epoch 1 Batch 400 Loss 1.6430 Acc0.5318
Epoch 1 Batch 500 Loss 1.5328 Acc0.5363
Epoch 1 Batch 600 Loss 1.4654 Acc0.5701
Epoch 1 Batch 700 Loss 1.4827 Acc0.5559
Epoch 1 Batch 800 Loss 1.3361 Acc0.5690
Epoch 1 Batch 900 Loss 1.2609 Acc0.6185
Epoch 1 Batch 1000 Loss 1.3951 Acc0.5880
Epoch 1 Batch 1100 Loss 1.3104 Acc0.5834
Epoch 1 Batch 1200 Loss 1.1884 Acc0.6061
Epoch 1 Batch 1300 Loss 1.1101 Acc0.6393
Epoch 1 Batch 1400 Loss 1.1736 Acc0.6053
Epoch 1 Batch 1500 Loss 1.0639 Acc0.6503
Epoch 1 Batch 1600 Loss 1.1868 Acc0.6142
Epoch 1 Batch 1700 Loss 1.1750 Acc0.6133
Epoch 1 Batch 1800 Loss 1.1203 Acc0.6118
Epoch 1 Batch 1900 Loss 1.3115 Acc0.5742
Epoch 1 Batch 2000 Loss 1.1153 Acc0.6292
Epoch 1 Batch 2100 Loss 1.0169 Acc0.6336
Epoch 1 Batch 2200 Loss 1.0494 Acc0.6221
Epoch 1 Batch 2300 Loss 1.0712 Acc0.6192
Epoch 1 Batch 2400 Loss 1.1009 Acc0.6342
Epoch 1 Batch 2500 Loss 1

KeyboardInterrupt: 

# Save Model 

In [24]:
x = datetime.datetime.now()
save_model_path = 'model_att_general_' + str(x)[:11]
print(f"Save model path is : {save_model_path}")

if not os.path.exists(save_model_path):
    os.makedirs(save_model_path)
    
encoder.save_weights(os.path.join(save_model_path, 'encoder.h5'))

decoder.save_weights(os.path.join(save_model_path, 'decoder.h5'))

with open(os.path.join(save_model_path,'tokenizer'+ str(num_decoder_tokens) ),'wb') as file:
    joblib.dump(tokenizer, file)

Save model path is : model_att_general_2022-06-09 


# Load Model

In [16]:
for ei,di, do in train:
    print(ei,di,do)
    break

save_model_path = './model_att_concat_2022-05-25 /'
encoder = Encoder(HIDDEN_DIM,time_steps_encoder,num_encoder_tokens)
output =encoder(ei,encoder.init_states(batch_size))
encoder.load_weights(os.path.join(save_model_path, 'encoder.h5'))

de_input = np.zeros((320, 1, 1500))
de_idx = tokenizer.word_index['bos']
de_state_h, de_state_c = output[1:]
decoder = Decoder(HIDDEN_DIM,time_steps_encoder,num_encoder_tokens)
decoder(de_input, (de_state_h, de_state_c), output[0])
decoder.load_weights(os.path.join(save_model_path, 'decoder.h5'))

/tmp/ipykernel_2265836/2721968490.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_sequences = np.array(train_sequences)


[[[0.         0.         1.819204   ... 0.40537566 0.         0.        ]
  [0.         0.         1.8118318  ... 1.0780952  0.         0.        ]
  [0.         0.         1.6541446  ... 1.4459221  0.         0.        ]
  ...
  [0.19641858 0.         2.2582536  ... 2.3994093  0.         0.8078662 ]
  [0.         0.         2.3914156  ... 2.899564   0.         0.5467725 ]
  [0.         0.         2.6686835  ... 2.4830456  0.         0.9339806 ]]

 [[0.         1.6346111  0.9470917  ... 0.         0.88411826 0.        ]
  [0.         1.418968   1.2187812  ... 0.         1.0428629  0.        ]
  [0.         1.4772139  1.2610791  ... 0.         0.8922333  0.        ]
  ...
  [1.0113212  0.         2.7485347  ... 0.         0.         0.21230459]
  [0.1960218  0.         2.6216874  ... 0.         0.         0.        ]
  [1.0169246  0.         3.0582728  ... 0.         0.1516946  0.        ]]

 [[1.3638706  3.9225347  0.         ... 0.         0.         0.46920353]
  [0.9628048  3.103214

2022-05-25 14:27:56.478665: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


# Inference

## 05. Inference Model Load

In [40]:
# class to perform inference on all test files and save as test_output.txt
class Video2Text(object):
    ''' Initialize the parameters for the model '''
    def __init__(self,encoder,decoder,tokenizer,save_model_path):
        self.latent_dim = 512
        self.num_encoder_tokens = 4096
        self.num_decoder_tokens = 1500
        self.time_steps_encoder = 80
        self.time_steps_decoder = None
        self.preload = True
        self.preload_data_path = 'preload_data'
        self.max_probability = -1

        # processed data
        self.encoder_input_data = []
        self.decoder_input_data = []
        self.decoder_target_data = []
        self.tokenizer = tokenizer

        # models
        self.encoder_model = None
        self.decoder_model = None
        self.inf_encoder_model = None
        self.inf_decoder_model = None
        self.save_model_path = save_model_path
        self.test_path = 'testing_data'
        self.inf_encoder_model = encoder
        self.decoder = decoder
        
    
    def decode_sequence2bs(self, input_seq):
        en_initial_states = self.inf_encoder_model.init_states(1)
        output, state_h,state_c = self.inf_encoder_model(input_seq,en_initial_states)
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, [state_h,state_c],[],[],0,output)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob,  path, lens,en_output):
        global decode_seq
        node = 2
        output_tokens, de_state_h, de_state_c, alignment = self.decoder(
                    target_seq, states_value, en_output)
        output_tokens = tf.reshape(output_tokens,self.num_decoder_tokens)#.reshape((self.num_decoder_tokens))

        sampled_token_index = tf.argsort(output_tokens,-1).numpy()[-10:][::-1]       
        states_value = [de_state_h, de_state_c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            if(sampled_char != 'eos' and lens <= config.max_length):
                p = output_tokens[sampled_token_index[i]]
                if(sampled_char == ''):
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)
                target_seq = np.zeros((1, 1, self.num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens+1,en_output)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if(p > self.max_probability):
                    decode_seq = path
                    self.max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        unigram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in unigram:
                unigram[c] += 1
            else:
                unigram[c] = 1
            if(last_string == c and idx2 > 0):
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def get_test_data(self, path):
        X_test = []
        X_test_filename = []
#         with open (os.path.join(path, 'testing_id.txt')) as testing_file:
#             lines = testing_file.readlines()
#             for filename in lines:
#                 filename = filename.strip()
#                 f = np.load(os.path.join(path , 'feat', filename + '.npy'))
#                 X_test.append(f)
#                 X_test_filename.append(filename[:-4])
#             X_test = np.array(X_test)
        for filename in test_l:
            try:
                f = np.load(os.path.join(path , 'features_dir', filename + '.npy'))
            except: 
                print('not exsits')
                continue
            X_test.append(f)
            X_test_filename.append(filename)
        return X_test, X_test_filename

    def test(self):
        X_test, X_test_filename = self.get_test_data(os.path.join(config.train_path)) 
        self.X_test , self.X_test_filename = X_test, X_test_filename
        # generate inference test outputs
        with open(os.path.join(self.save_model_path, 'test_output_topk.txt'), 'w') as file:
            for idx, x in enumerate(X_test): 
                file.write(X_test_filename[idx]+',')
                decoded_sentence = self.decode_sequence2bs(x.reshape(-1, 80, 4096))
                decode_str = self.decoded_sentence_tuning(decoded_sentence)
                for d in decode_str:
                    file.write(d + ' ')
                file.write('\n')
                # re-init max prob
                self.max_probability = -1
                
    def index_to_word(self):
        # inverts word tokenizer
        index_to_word = {value: key for key, value in self.tokenizer.word_index.items()}
        return index_to_word
                
    def greedy_search(self, f):
        """
                :param f: the loaded numpy array after creating videos to frames and extracting features
                :return: the final sentence which has been predicted greedily
                """
        inv_map = self.index_to_word()
        encoder_output = self.inf_encoder_model.predict(f.reshape(-1, 80, 4096))
        target_seq = np.zeros((1, 1, 1500))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(config.max_length):
            output_tokens = self.inf_decoder_model.predict([target_seq, encoder_output])
            output_tokens = output_tokens.reshape(self.num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, 1500))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

In [ ]:
c = Video2Text(encoder,decoder,tokenizer,save_model_path)
c.test()